In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import scipy.signal 
import scipy.ndimage 
from scipy.interpolate import UnivariateSpline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

In [ ]:
def get_dataset(func, n_samples=1024):
    x = np.linspace(-10, 10, n_samples).astype(np.float32)
    y = func(x).astype(np.float32)
    y = y.reshape(1, -1) 
    return torch.from_numpy(y).to(device), torch.from_numpy(x).to(device)

In [ ]:
# import diffusers.schedulers.scheduling_ddpm 

# def get_beta_schedule(T):
#     return torch.linspace(1e-4, 0.02, T).to(device)

def betas_list(T, kind='cosine'):
    if kind == 'cosine':
        steps = torch.arange(T, dtype=torch.float32, device=device)
        betas = (torch.cos(steps / T * (torch.pi / 2))) ** 2 * 0.01
        return betas
    elif kind == 'exp':
        start = torch.log(torch.tensor(1e-4, device=device))
        end = torch.log(torch.tensor(0.01, device=device))
        betas = torch.exp(torch.linspace(start, end, T, device=device))
        return betas
    else:
        return torch.linspace(1e-4, 0.01, T, device=device)



# def get_beta_schedule(beta_schedule, *, beta_start, beta_end, num_diffusion_timesteps): #TODO hugging face
#   if beta_schedule == 'quad':
#     betas = np.linspace(beta_start ** 0.5, beta_end ** 0.5, num_diffusion_timesteps, dtype=np.float64) ** 2
#   elif beta_schedule == 'linear':
#     betas = np.linspace(beta_start, beta_end, num_diffusion_timesteps, dtype=np.float64)
#   elif beta_schedule == 'warmup10':
#     betas = _warmup_beta(beta_start, beta_end, num_diffusion_timesteps, 0.1)
#   elif beta_schedule == 'warmup50':
#     betas = _warmup_beta(beta_start, beta_end, num_diffusion_timesteps, 0.5)
#   elif beta_schedule == 'const':
#     betas = beta_end * np.ones(num_diffusion_timesteps, dtype=np.float64)
#   elif beta_schedule == 'jsd':  # 1/T, 1/(T-1), 1/(T-2), ..., 1
#     betas = 1. / np.linspace(num_diffusion_timesteps, 1, num_diffusion_timesteps, dtype=np.float64)
#   else:
#     raise NotImplementedError(beta_schedule)
#   assert betas.shape == (num_diffusion_timesteps,)
#   return betas

In [ ]:
def forward_diffusion_sample(x0, t, betas):
    noise = torch.randn_like(x0)
    sqrt_alpha_cumprod = torch.sqrt(torch.cumprod(1. - betas, dim=0)).to(device)
    alpha_t = sqrt_alpha_cumprod[t].view(-1, 1)
    xt = alpha_t * x0 + torch.sqrt(1 - alpha_t ** 2) * noise
    return xt, noise

In [ ]:
class UNet1D(nn.Module):
    def __init__(self, in_channels=1, base_channels=32, signal_length=1024, time_emb_dim=64):
        super().__init__()

        self.time_embed = nn.Sequential(
            nn.Embedding(1000, time_emb_dim),          
            nn.Linear(time_emb_dim, signal_length)       
        )

        self.downs = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(in_channels, base_channels, kernel_size=4, stride=2, padding=1),
                nn.ReLU()
            ),
            nn.Sequential(
                nn.Conv1d(base_channels, base_channels*2, kernel_size=4, stride=2, padding=1),
                nn.ReLU()
            )
        ])
        self.mid = nn.Sequential(
            nn.Conv1d(base_channels*2, base_channels*2, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.ups = nn.ModuleList([
            nn.Sequential(
                nn.ConvTranspose1d(base_channels*2, base_channels, kernel_size=4, stride=2, padding=1),
                nn.ReLU()
            ),
            nn.Sequential(
                nn.ConvTranspose1d(base_channels, in_channels, kernel_size=4, stride=2, padding=1)
            )
        ])
        
    def forward(self, x, t):
        x = x.unsqueeze(1) 

        t_emb = self.time_embed(t)         
        t_emb = t_emb.unsqueeze(1)         
        x = x + t_emb                     

        h1 = self.downs[0](x)
        h2 = self.downs[1](h1)
        h = self.mid(h2)
        h = self.ups[0](h)
        h = self.ups[1](h)
        return h.squeeze(1)  

In [ ]:
def train(model, dataset, betas, T=100, epochs=1000, log_every=100, loss_fn=F.mse_loss):
    model.train()
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(epochs):
        for (x0,) in dataloader:
            t = torch.randint(0, T, (x0.size(0),), device=device)
            xt, noise = forward_diffusion_sample(x0, t, betas)
            pred = model(xt, t)
            loss = loss_fn(pred, noise)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # if epoch % log_every == 0 or epoch == epochs - 1:
        #     print(f"Epoch {epoch+1}/{epochs} | Loss: {loss.item():.6f}")

In [ ]:
# def compare_signals(original, test, label):
#     mse = F.mse_loss(test, original).item()

#     print(f"{label} mse: {mse:.6f}")

In [ ]:
@torch.no_grad()
def diff_process(model, func, betas, steps=100, x=None):
    model.eval()
    if x is None:
        x = np.linspace(-10, 10, 1024).astype(np.float32)
    y = func(x).astype(np.float32)
    y_torch = torch.from_numpy(y).to(device).view(1, -1)

    noisy_versions = []
    noise = torch.randn_like(y_torch)
    sqrt_alpha_cumprod = torch.sqrt(torch.cumprod(1 - betas, dim=0)).to(device)

    for t in range(steps):
        alpha_bar = sqrt_alpha_cumprod[t]
        xt = alpha_bar * y_torch + torch.sqrt(1 - alpha_bar**2) * noise
        noisy_versions.append(xt.detach().cpu().numpy()[0])

    recovered_versions = []
    xt = noisy_versions[-1]
    xt = torch.from_numpy(xt).to(device).view(1, -1)

    for t in reversed(range(steps)):
        t_tensor = torch.tensor([t], device=device)
        pred_noise = model(xt, t_tensor)
        coef = 1 / torch.sqrt(1 - betas[t])
        xt = (xt - betas[t] * pred_noise) * coef
        recovered_versions.append(xt.detach().cpu().numpy()[0])

    indices_to_show = [int(steps * i / 10) for i in range(11)]
    if indices_to_show[-1] != steps-1:
        indices_to_show[-1] = steps-1

    fig, axs = plt.subplots(2, 11, figsize=(33, 6), sharey=True)

    for i, idx in enumerate(indices_to_show):
        axs[0, i].plot(x, y, label='Oryginał', linewidth=1, color='black')
        axs[0, i].plot(x, noisy_versions[idx], label=f"Krok {idx}", color='blue')
        axs[0, i].set_title(f"Zaszum {idx}")
        axs[0, i].set_xticks([])
        axs[0, i].set_yticks([])

        axs[1, i].plot(x, y, label='Oryginał', linewidth=1, color='black')
        axs[1, i].plot(x, recovered_versions[idx], label=f"Odszum {idx}", color='red')
        axs[1, i].set_title(f"Odszum {idx}")
        axs[1, i].set_xticks([])
        axs[1, i].set_yticks([])

    axs[0, 0].legend(loc='upper right')

    original = torch.from_numpy(y).to(device).view(1, -1)
    noisy_final = torch.from_numpy(noisy_versions[-1]).to(device).view(1, -1)
    denoised_final = torch.from_numpy(recovered_versions[-1]).to(device).view(1, -1)

    # compare_signals(original, noisy_final, label='zaszumiony (ostatni krok)')
    # compare_signals(original, denoised_final, label='odszumiony (ostatni krok)')

    original_np = original.cpu().numpy()
    denoised_np = denoised_final.cpu().numpy()
    noisy_final_np = noisy_final.cpu().numpy()
    x_np =  x # x.cpu().numpy()

    mse_noisy = np.mean((original_np - noisy_final_np) ** 2)
    rmse_noisy = np.sqrt(mse_noisy) 
    mae_noisy = np.mean(np.abs(original_np - noisy_final_np))
    mape_noisy = np.mean(np.abs((original_np - noisy_final_np) / (original_np + 1e-8))) * 100
    # rse_noisy = np.sum((original_np - noisy_final_np) ** 2) / (np.sum((original_np - np.mean(original_np)) ** 2) + 1e-8)

    mse = np.mean((original_np - denoised_np) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(original_np - denoised_np))
    mape = np.mean(np.abs((original_np - denoised_np) / (original_np + 1e-8))) * 100
    # rse = np.sum((original_np - denoised_np) ** 2) / (np.sum((original_np - np.mean(original_np)) ** 2) + 1e-8)

    # print(f"original-noisy ----- MSE: {mse_noisy:.6f}; RMSE: {rmse_noisy:.6f}; MAE: {mae_noisy:.6f}; MAPE: {mape_noisy:.6f}%")

    # print(f"original-denoised ----- MSE: {mse:.6f}; RMSE: {rmse:.6f}; MAE: {mae:.6f}; MAPE: {mape:.6f}%")


    plt.tight_layout()
    plt.show()

    fig_smooth, axs_smooth = plt.subplots(1, 3, figsize=(14, 4), sharex=True, sharey=True) 
    axs_smooth[0].plot(x_np, original_np[0], label='Oryginał', color='black')
    axs_smooth[0].plot(x_np, denoised_np[0], label='Odszumiony', color='red')
    axs_smooth[0].set_title('Oryginał vs odszumiony')
    axs_smooth[0].legend()

    sigma_gaussian = 5 
    smoothed_gaussian = scipy.ndimage.gaussian_filter1d(denoised_np[0], sigma=sigma_gaussian)
    axs_smooth[1].plot(x_np, original_np[0], label='Oryginał', color='black')
    axs_smooth[1].plot(x_np, smoothed_gaussian, label=f'Wygładzony (Gauss, $\\sigma$={sigma_gaussian})', color='green')
    axs_smooth[1].set_title(f'Wygładzenie Gaussem ($\sigma$={sigma_gaussian})')
    axs_smooth[1].legend()

    spline_s_param = 0.8 
    spl = UnivariateSpline(x_np, denoised_np[0], s=spline_s_param)
    smoothed_spline = spl(x_np)
    axs_smooth[2].plot(x_np, original_np[0], label='Oryginał', color='black')
    axs_smooth[2].plot(x_np, smoothed_spline, label=f'Aproksymacja splajnem (s={spline_s_param})', color='purple')
    axs_smooth[2].set_title(f'Aproksymacja splajnem (s={spline_s_param})')
    axs_smooth[2].legend()

    plt.tight_layout()
    plt.show()

    mse_gaussian = np.mean((original_np[0] - smoothed_gaussian) ** 2)
    mse_spline = np.mean((original_np[0] - smoothed_spline) ** 2)

    # print(f"MSE po Gaussie: {mse_gaussian:.6f}")
    # print(f"MSE po splajach: {mse_spline:.6f}")

    return {
       'mse_noisy': mse_noisy, 
        'rmse_noisy': rmse_noisy,
        'mae_noisy': mae_noisy,
        'mape_noisy': mape_noisy,
        # 'rse_noisy': rse_noisy,

        'mse_denoised': mse, 
        'rmse_denoised': rmse,
        'mae_denoised': mae,
        'mape_denoised': mape,
        # 'rse_denoised': rse,

        'mse_gaussian': mse_gaussian,
        'mse_spline': mse_spline
    }


In [ ]:
def f_sin(x): return np.sin(x) 
def f_tan(x): return np.tan(x)
def f_sgn(x): return np.sign(x)
def f_sigmoid(x): return 1 / (1 + np.exp(-x))
def f_relu(x): return np.maximum(0, x)
def f_log10(x): return np.log10(np.clip(x, 1e-3, None))
def f_log2(x): return np.log2(np.clip(x, 1e-3, None))
def f_inv(x): return 1 / np.clip(x, 1e-3, None)
def f_exp(x): return np.exp(x)
def f_poly(x): return x**2 + 2*x + 1
def f_sin_1x(x): return np.sin(1/x)
def f_sin_2(x): return np.sin(x)**2

f_names =  [("sin", f_sin),("exp", f_exp),("log10", f_log10),("poly", f_poly),
            ("sigmoid", f_sigmoid),("relu", f_relu),("inv", f_inv),("sin_1x", f_sin_1x),("zin_2", f_sin_2)] #("log2", f_log2),


# f_names =  [("log10", f_log10),("log2", f_log2),("poly", f_poly)]


In [ ]:
import json
import os

def save_result(result, path="results10000.json"):
    for key, value in result.items():
        if isinstance(value, np.ndarray):
            result[key] = value.tolist()  
        elif isinstance(value, np.float32):
            result[key] = float(value)  

    results = []
    if os.path.exists(path):
        with open(path, "r") as f:
            try:
                content = f.read().strip()
                if content: 
                    results = json.loads(content)
                else:
                    results = [] 
            except json.JSONDecodeError:
                results = [] 
    results.append(result)
    with open(path, "w") as f:
        json.dump(results, f, indent=4)


In [ ]:
def rmse_loss(pred, target):
    return torch.sqrt(F.mse_loss(pred, target))

def mape_loss(pred, target, epsilon=1e-8): #bo nie wolno przez 0
    return torch.mean(torch.abs((target - pred) / (target + epsilon))) * 100

In [ ]:
if __name__ == '__main__':
    log_every = 200

    epochs = [10000]
    Ts = [100, 350, 800]

    loss_functions_for_training = {
        "MSE": F.mse_loss,
    }

    for epoch in epochs:
        for ts in Ts:
            betas_cosine = betas_list(ts, kind='cosine')
            betas_exp = betas_list(ts, kind='exp')
            betas_linear = betas_list(ts, kind='linear')
            betas = [("cosine", betas_cosine), ("exp", betas_exp), ("linear", betas_linear)]
            for beta_name, beta in betas:
                for loss_name_train, current_loss_fn_train in loss_functions_for_training.items():
                    for func_name, func_fn in f_names: 
                        y, x = get_dataset(func_fn)
                        dataset = TensorDataset(y)

                        model = UNet1D().to(device)

                        print(f"-----------function {func_name}---------, epochs: {epoch}, T: {ts}, beta type:{beta_name}, training_loss: {loss_name_train}-----------")
                        train(model, dataset, beta, T=ts, epochs=epoch, log_every=log_every, loss_fn=current_loss_fn_train)

                        metrics = diff_process(model, func_fn, beta, steps=ts, x=x.cpu().numpy())
                        result = {
                            "function": func_name,
                            "beta": beta_name,
                            "T": ts,
                            "epochs": epoch,
                            "training_loss_function": loss_name_train, 
                            **metrics
                        }
                        save_result(result)


In [ ]:
if __name__ == '__main__':
    log_every = 200

    epochs = [10000]
    Ts = [100, 350, 800]

    loss_functions_for_training = {
        "RMSE": rmse_loss
    }

    for epoch in epochs:
        for ts in Ts:
            betas_cosine = betas_list(ts, kind='cosine')
            betas_exp = betas_list(ts, kind='exp')
            betas_linear = betas_list(ts, kind='linear')
            betas = [("cosine", betas_cosine), ("exp", betas_exp), ("linear", betas_linear)]
            for beta_name, beta in betas:
                for loss_name_train, current_loss_fn_train in loss_functions_for_training.items():
                    for func_name, func_fn in f_names: 
                        y, x = get_dataset(func_fn)
                        dataset = TensorDataset(y)

                        model = UNet1D().to(device)

                        print(f"-----------function {func_name}---------, epochs: {epoch}, T: {ts}, beta type:{beta_name}, training_loss: {loss_name_train}-----------")
                        train(model, dataset, beta, T=ts, epochs=epoch, log_every=log_every, loss_fn=current_loss_fn_train)

                        metrics = diff_process(model, func_fn, beta, steps=ts, x=x.cpu().numpy())
                        result = {
                            "function": func_name,
                            "beta": beta_name,
                            "T": ts,
                            "epochs": epoch,
                            "training_loss_function": loss_name_train, 
                            **metrics
                        }
                        save_result(result)


In [ ]:
if __name__ == '__main__':
    log_every = 200

    epochs = [10000]
    Ts = [100, 350, 800]

    loss_functions_for_training = { 
        "MAE": F.l1_loss
    }

    for epoch in epochs:
        for ts in Ts:
            betas_cosine = betas_list(ts, kind='cosine')
            betas_exp = betas_list(ts, kind='exp')
            betas_linear = betas_list(ts, kind='linear')
            betas = [("cosine", betas_cosine), ("exp", betas_exp), ("linear", betas_linear)]
            for beta_name, beta in betas:
                for loss_name_train, current_loss_fn_train in loss_functions_for_training.items():
                    for func_name, func_fn in f_names: 
                        y, x = get_dataset(func_fn)
                        dataset = TensorDataset(y)

                        model = UNet1D().to(device)

                        print(f"-----------function {func_name}---------, epochs: {epoch}, T: {ts}, beta type:{beta_name}, training_loss: {loss_name_train}-----------")
                        train(model, dataset, beta, T=ts, epochs=epoch, log_every=log_every, loss_fn=current_loss_fn_train)

                        metrics = diff_process(model, func_fn, beta, steps=ts, x=x.cpu().numpy())
                        result = {
                            "function": func_name,
                            "beta": beta_name,
                            "T": ts,
                            "epochs": epoch,
                            "training_loss_function": loss_name_train, 
                            **metrics
                        }
                        save_result(result)


In [ ]:
if __name__ == '__main__':
    log_every = 200

    epochs = [10000]
    Ts = [100, 350, 800]

    loss_functions_for_training = {
        "MAPE": mape_loss
    }

    for epoch in epochs:
        for ts in Ts:
            betas_cosine = betas_list(ts, kind='cosine')
            betas_exp = betas_list(ts, kind='exp')
            betas_linear = betas_list(ts, kind='linear')
            betas = [("cosine", betas_cosine), ("exp", betas_exp), ("linear", betas_linear)]
            for beta_name, beta in betas:
                for loss_name_train, current_loss_fn_train in loss_functions_for_training.items():
                    for func_name, func_fn in f_names: 
                        y, x = get_dataset(func_fn)
                        dataset = TensorDataset(y)

                        model = UNet1D().to(device)

                        print(f"-----------function {func_name}---------, epochs: {epoch}, T: {ts}, beta type:{beta_name}, training_loss: {loss_name_train}-----------")
                        train(model, dataset, beta, T=ts, epochs=epoch, log_every=log_every, loss_fn=current_loss_fn_train)

                        metrics = diff_process(model, func_fn, beta, steps=ts, x=x.cpu().numpy())
                        result = {
                            "function": func_name,
                            "beta": beta_name,
                            "T": ts,
                            "epochs": epoch,
                            "training_loss_function": loss_name_train, 
                            **metrics
                        }
                        save_result(result)
